In [1]:
import time
starttime = time.time()
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import GPy

import pandas as pd
from gpr4im import pk_tools as pk
from gpr4im import fg_tools as fg

Image = fits.open('/mnt/sdc3a.MS.0/sdc3dataset/Image/ZW3.msn_image.fits')
data = Image[0].data
Image.close()


data_r= np.transpose(data, (1, 2, 0))


In [2]:
def pt(text):
    file = open(r"Logs/Log27_07_2023_22_10.txt","a+")
    file.write(text)
    file.write('\n')
    file.close()
    return

def System_status():
    # Importing the library
    import time
    import psutil
    # Getting usage of virtual_memory in GB ( 4th field)
    Mem = psutil.virtual_memory()[3]/1000000000
    pt(f'RAM Used (GB):{Mem}')
    endtime = time.time()
    hours=(endtime-starttime)/3600
    pt(f"Time taken {hours} Hr") 
    print(f"Time taken {hours} Hr") 
    return

def Type_Size(Input_variable):
    import sys
    
    pt(type(Input_variable))
    pt(np.shape(Input_variable))
    pt(sys.getsizeof(Input_variable))
    return

def BrightnessTemp (Intensity_Data_Cube,Start_frequency,Total_subbands):

    pt("BrightnessTemp:")
    #beam discription for the SKASDCa project
    Bmaj = 0.0553607952110976
    Bmin = 0.0411477702279064
    t_maj = Bmaj*np.pi/180
    t_min = Bmin*np.pi/180
    T_data = np.zeros(np.shape(Intensity_Data_Cube))
    C=1222/(t_maj*t_min)
    for i in range (Total_subbands):
        f_l = i*0.1*1e6
        v = Start_frequency+f_l
        T_data[:,:,i]=(C/(v*v))*Intensity_Data_Cube[:,:,i]
            
    pt("Convertion completed.")

    return T_data

def frange(fmin,fmax):
    df = (fmax-fmin)/150
    return np.arange(fmin,fmax,df)

def calculate_redshift(Omega_m, Omega_lambda, H0):
    # Calculate the critical density
    rho_critical = (3 * (H0**2)) / (8 * np.pi * 6.67430e-11)
    
    # Calculate the matter density
    rho_m = Omega_m * rho_critical
    
    # Calculate the dark energy density
    rho_lambda = Omega_lambda * rho_critical
    
    # Calculate the redshift
    redshift = ((rho_lambda / rho_m) ** (1/3)) - 1
    
    return redshift


In [3]:
Data1=data_r[:,:,0:150]
Data2=data_r[:,:,150:300]
Data3=data_r[:,:,300:450]
Data4=data_r[:,:,450:600]
Data5=data_r[:,:,600:750]
Data6=data_r[:,:,750:900]

In [4]:
np.shape(Data6)

(2048, 2048, 150)

In [5]:
freqs1 = frange(106,121)
freqs2 = frange(121,136)
freqs3 = frange(136,151)
freqs4 = frange(151,166)
freqs5 = frange(166,181)
freqs6 = frange(181,196)

In [6]:
# kernel for the smooth foreground:
kern_fg = GPy.kern.RBF(1)
# kernel for the HI cosmological signal:
kern_21 = GPy.kern.Exponential(1)

In [7]:
# setting priors for the smooth foreground kernel:
kern_fg.variance.constrain_bounded(1000,100000000)
kern_fg.lengthscale.constrain_bounded(200,10000)
# setting priors for the HI kernel:
kern_21.variance.constrain_bounded(0.000001,0.5)
kern_21.lengthscale.constrain_bounded(0.01,15)

reconstraining parameters rbf.variance
reconstraining parameters rbf.lengthscale
reconstraining parameters Exponential.variance
reconstraining parameters Exponential.lengthscale


In [8]:
kern = kern_fg + kern_21

In [9]:
num_restarts = 1
NprePCA = 0
noise_data = None
zero_noise = True
heteroscedastic = False
invert = False

In [10]:

Omega_m = 0.30964
Omega_lambda = 0.69036
H0 = 100

redshift = calculate_redshift(Omega_m, Omega_lambda, H0)
print("Redshift:", redshift)

Redshift: 0.3063900192746536


In [11]:

import numpy as np
import tools21cm as t2c

A=  [9.0133,9.0133,15]
Z =redshift#0.3063900192746536
X  = t2c.cosmology.deg_to_cdist(A,Z)

print(X)

[193.00139342 193.00139342 321.19433519]


In [12]:
# Grid dimension of data cube:
#lx=F
lx,ly,lz = X#Mpc/h
nx,ny,nz = 2048,2048,150
# weighting and window function:
w = W = np.ones((nx,ny,nz))
# number of k bins:
nkbin = 10
# setting array of k bin edges:
kbins = [5.0e-02,1.0e-01,1.5e-01,2.0e-01,2.5e-01,3.0e-01,3.5e-01,4.0e-01,4.5e-01,5.0e-01]
# setting array of k bin centers"
k = 5.0e-02

In [13]:
DATA=[Data1,Data2,Data3,Data4,Data5,Data6]
F_R=[freqs1,freqs2,freqs3,freqs4,freqs5,freqs6]
S_Freq = [106e6,121e6,136e6,151e6,166e6,181e6]
Subbands = 150 #(15MHz *10subbands of 100KHz perMHz)
S = ['Output/SROT_106.0MHz-121.0MHz.data','Output/SROT_121.0MHz-136.0MHz.data','Output/SROT_136.0MHz-151.0MHz.data',
    'Output/SROT_151.0MHz-166.0MHz.data','Output/SROT_166.0MHz-181.0MHz.data','Output/SROT_181.0MHz-196.0MHz.data']
E = ['Output/SROT_106.0MHz-121.0MHz_errors.data','Output/SROT_121.0MHz-136.0MHz_errors.data','Output/SROT_136.0MHz-151.0MHz_errors.data',
    'Output/SROT_151.0MHz-166.0MHz_errors.data','Output/SROT_166.0MHz-181.0MHz_errors.data','Output/SROT_181.0MHz-196.0MHz_errors.data']



In [14]:
import gc
System_status()
D = np.zeros(np.shape(DATA[0])) 
Gpr_Result =0
Spect = np.zeros((9,9))
Nmatrix = np.zeros((9,9))
Arr= np.zeros((2,9,9))
for i in range(len(DATA)):
    Arr= np.zeros((2,9,9))
    for j in range(2):
        
        print(f"Start of {i}{j} th loop")
        pt(f"Start of {i}{j} th loop")
        System_status()
        Gpr_Result = fg.GPRclean(DATA[i], F_R[i], kern_fg, kern_21, NprePCA=NprePCA, num_restarts=num_restarts,
                                             noise_data=noise_data, heteroscedastic=heteroscedastic, 
                                             zero_noise=zero_noise, invert=invert)
    
        D = BrightnessTemp(Gpr_Result.res,S_Freq[i],Subbands)
        Spect,Nmatrix= pk.PerpParaPk(D,nx,ny,nz,lx,ly,lz,kbins,kbins,w,W)
        Arr[j] = Spect
        pt(f"F={S_Freq[i]} \n Spectrum{Spect}")
        print(f"End of {i}{j} th loop")
        pt(f"End of {i}{j} th loop")
        del D
        del Gpr_Result 
        del Spect
        del Nmatrix
        
        System_status()
    spect = Arr.mean(axis=0)
    np.savetxt(S[i],spect,delimiter = ' ')
    spect_error = Arr.std(axis=0)
    np.savetxt(E[i],spect_error,delimiter = ' ')
    del Arr
    print(f"End main of {i} th loop")
    pt(f"End main of {i} th loop")
    del spect
    del spect_error
    System_status()
    gc.collect()
System_status()   

Time taken 0.0017442482709884644 Hr
Start of 00 th loop
Time taken 0.0017447336514790854 Hr
Optimization restart 1/1, f = -3511601746.2305126

Calculating P(k_perp,k_para)...
End of 00 th loop
Time taken 0.8538006748755773 Hr
Start of 01 th loop
Time taken 0.8538008456097709 Hr
Optimization restart 1/1, f = -3511601746.2305126

Calculating P(k_perp,k_para)...
End of 01 th loop
Time taken 1.714610135157903 Hr
End main of 0 th loop
Time taken 1.7146111992994943 Hr
Start of 10 th loop
Time taken 1.714711573521296 Hr
Optimization restart 1/1, f = -3591458487.9252186

Calculating P(k_perp,k_para)...
End of 10 th loop
Time taken 2.648860153887007 Hr
Start of 11 th loop
Time taken 2.6488603762785594 Hr
Optimization restart 1/1, f = -3591458487.9252186

Calculating P(k_perp,k_para)...
End of 11 th loop
Time taken 3.564819830854734 Hr
End main of 1 th loop
Time taken 3.564820890360408 Hr
Start of 20 th loop
Time taken 3.564925046695603 Hr
Optimization restart 1/1, f = -3632850356.985003

Calcul